# Generate country profiles

Documenting the generation of country profiles

### Load necessary libraries

In [1]:
import csv
import json
import urllib3  # allows to access a URL with python
import urllib
import re
import statistics
import math
import os

### User parameters

In [2]:
os.chdir('C:\\Users\\L.GonzalezMorales\\Documents\\GitHub\\FIS4SDGs\\unsd\\country-profiles\\ProjectCountryProfilesPython\\') 
           
release = '2019.Q1.G.02' # Make sure to have the correct release here


## Auxiliary funtions

### Commercial rounding

In [3]:
def round_KFM(x, n):
    posneg = math.copysign(1, x)
    z = abs(x)*10**n
    z = z + 0.5
    z = math.trunc(z)
    z = z/10**n
    result = z * posneg
    return result
    


In [4]:
print(round_KFM(3.5023,2))

3.5


#### Regular expression to capture numeric values (including those in scientific notation)
The regex is

```
-?      # an optional -
\d+     # a series of digits
(?:     # start non capturing group
  \.    # a dot
  \d+   # a series of digits
)?      
(?:     # start non capturing group
  e     # "e"
  -?    # an optional -
  \d+   # digits
)?
```


In [5]:
def numeric_part(v):
    numeric_part_f = re.compile(r'-?\d+(?:\.\d+)?(?:e-?\d+)?')
    x = numeric_part_f.findall(v)
    if len(x) > 0:
        return float(x[0])
    else:
        return None


In [6]:
print(numeric_part('<4'))
print(numeric_part('abs'))
print(numeric_part('-23.2e2'))
print(numeric_part('-23.2e2 or more'))

4.0
None
-2320.0
-2320.0


In [7]:
def non_numeric_part(v):
    numeric_part_f = re.compile(r'-?\d+(?:\.\d+)?(?:e-?\d+)?')
    x = numeric_part_f.findall(v)
    if len(x) > 0:
        return v.replace(x[0],'')
    else:
        return v


In [8]:
print(non_numeric_part('<4'))
print(non_numeric_part('abs'))
print(non_numeric_part('-23.2e2'))
print(non_numeric_part('xsy-23.2e2 or more'))

<
abs

xsy or more


#### Disable insecure request warnings when using `urllib3`.

In [9]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

## Main steps

### (1) Read fact-builder conditions

In [10]:
fact_builder = []
with open('CountryProfileBuilder.txt', newline = '') as countryProfileBuilder:                                                                                          
    country_profile_builder = csv.DictReader(countryProfileBuilder, delimiter='\t')
    for row in country_profile_builder:
        fact_builder.append(dict(row))
        
fact_builder[1]


{'No.': '2',
 'SDMX_CODE': 'SI_POV_EMP1',
 'SeriesCode': 'SI_POV_EMP1',
 'HubItemId': 'deb1f4a7e2e14d78a113c53cba2838de',
 'Profile.Series': '1',
 'Goal.ID': '1',
 'Goal.Name': 'Goal 1: End poverty in all its forms everywhere',
 'P.IndicatorCode': 'C010101',
 'P.IndicatorName.2017': 'Employed population below international poverty line',
 'Age': '15+',
 'Location': '',
 'Sex': 'BOTHSEX',
 'Bounds': '',
 'Education level': '',
 'Type of product': '',
 'Type of mobile technology': '',
 'Type of speed': '',
 'zero.removal': '1',
 'Text.type': '2',
 'unit1': '% ',
 'unit2': '%',
 'DA2.1': " of the country's workers and their families were living on less than 1.90 US dollars per person per day",
 'DA2.2': '',
 'DA3.1': " of the country's workers and their families were living on less than 1.90 US dollars per person per day",
 'DA3.2': '',
 'Down': ' down from ',
 'Up': ' up from '}

### (2) Create list of countries

In [11]:
countryArray = []
with open('CountryListXY.txt', newline = '') as countryListXY:                                                                                          
    countryListXY = csv.DictReader(countryListXY, delimiter='\t')
    for row in countryListXY:
        countryArray.append(dict(row))
        
    
print("Total number or countries: ", len(countryArray))
countryArray[0:1]


Total number or countries:  248


[{'X': '66.02688198',
  'Y': '33.83160199',
  'ISO3CD': 'AFG',
  'geoAreaCode': '4',
  'geoAreaName': 'Afghanistan',
  'UNMember': '1',
  'CountryProfile': '1'}]

### (3) Get the list of goals, targets and indicators

##### 3.1 Start by creating a `PoolManager()` object using `urllib3`

In [12]:
http = urllib3.PoolManager()

response = http.request('GET', "https://unstats.un.org/SDGAPI/v1/sdg/Goal/List?includechildren=true")
indicator_framework = json.loads(response.data.decode('UTF-8'))

#print(indicator_framework[0])

In [13]:
goal_temp = indicator_framework[3]
goal_temp.keys()
print(goal_temp['code'])
print(goal_temp['title'])
print(goal_temp['description'])


4
Ensure inclusive and equitable quality education and promote lifelong learning opportunities for all
Goal 4 focuses on the acquisition of foundational and higher-order skills; greater and more equitable access to technical and vocational education and training and higher education; training throughout life; and the knowledge, skills and values needed to function well and contribute to society.


In [14]:
print(goal_temp['targets'][0].keys())
print(goal_temp['targets'][0]['title'])
print(goal_temp['targets'][0]['description'])



dict_keys(['goal', 'code', 'title', 'description', 'uri', 'indicators'])
By 2030, ensure that all girls and boys complete free, equitable and quality primary and secondary education leading to relevant and effective learning outcomes
By 2030, ensure that all girls and boys complete free, equitable and quality primary and secondary education leading to relevant and effective learning outcomes


In [15]:
goal_temp['targets'][0]['indicators'][0].keys()

dict_keys(['goal', 'target', 'code', 'description', 'tier', 'uri', 'series'])

##### 3.2 Flatten into a list of dictionaries

In [16]:

series_list = []

keys = ["goalCode", 
        "goalDesc",
        "targetCode",
        "targetDesc",
        "indicatorCode",
        "indicatorDesc",
        "indicatorTier",
        "seriesCode",
        "seriesDesc"
       ]

for g in indicator_framework:
    for t in g['targets']:
        for i in t['indicators']:
            for s in i['series']:
                if s['release'] == release:
                    values = [g['code'], g['title'],
                              t['code'], t['description'], 
                              i['code'], i['description'], i['tier'], 
                              s['code'], s['description']]
                    
                    keys_and_values = zip(keys, values)
                    serie_dic = {}
                    for key, value in keys_and_values:
                        serie_dic[key] = value
                    series_list.append(serie_dic)
                        

series_list[0]

{'goalCode': '1',
 'goalDesc': 'End poverty in all its forms everywhere',
 'targetCode': '1.1',
 'targetDesc': 'By 2030, eradicate extreme poverty for all people everywhere, currently measured as people living on less than $1.25 a day',
 'indicatorCode': '1.1.1',
 'indicatorDesc': 'Proportion of population below the international poverty line, by sex, age, employment status and geographical location (urban/rural)',
 'indicatorTier': '1',
 'seriesCode': 'SI_POV_DAY1',
 'seriesDesc': 'Proportion of population below international poverty line (%)'}

### (4) Create country profiles

#### Get slice description

In [17]:
def get_slice_description(goalCode,seriesCode, seriesList):
    
   # slice_description = None
    
    for s in series_list:
        if (s['goalCode']==goalCode and s['seriesCode'] == seriesCode):
            slice_description = s
            continue
    
    return slice_description

In [18]:
get_slice_description('12','EN_MAT_DOMCMPC',series_list )

{'goalCode': '12',
 'goalDesc': 'Ensure sustainable consumption and production patterns',
 'targetCode': '12.2',
 'targetDesc': 'By 2030, achieve the sustainable management and efficient use of natural resources',
 'indicatorCode': '12.2.2',
 'indicatorDesc': 'Domestic material consumption, domestic material consumption per capita, and domestic material consumption per GDP',
 'indicatorTier': '1',
 'seriesCode': 'EN_MAT_DOMCMPC',
 'seriesDesc': 'Domestic material consumption per capita, by type of raw material (tonnes)'}

#### Get slice dimensions

In [19]:
def get_slice_dimensions(this_fact):

    slice_dimensions = {d: this_fact[d] for d in this_fact.keys() &
                        {'Age', 
                         'Location', 
                         'Sex', 
                         'Bounds', 
                         'Education level', 
                         'Type of product', 
                         'Type of mobile technology', 
                         'Type of speed'}}


    slice_dimensions = dict((k, v) for k, v in slice_dimensions.items() if v != '')

    return slice_dimensions


In [20]:
get_slice_dimensions(fact_builder[1])

{'Age': '15+', 'Sex': 'BOTHSEX'}

#### Collect fact data

In [21]:
def get_fact_data(this_fact, country_code):
    
    slice_dimensions = get_slice_dimensions(this_fact)
    
    series = this_fact['SeriesCode']

    baseURL = 'https://unstats.un.org/SDGAPI/v1/sdg/Series/'

    if(len(slice_dimensions)>0):

        dimensions = ""
        first = True
        for d in slice_dimensions:
            dimension_name = d
            dimension_value = slice_dimensions[d]
            dimension_string = '{name:"'+ dimension_name +'",values:["'+ dimension_value + '"]}'
            if first:
                dimensions = dimension_string
                first = False
            else:
                dimensions = dimensions + "," + dimension_string

        dimensions = "[" + dimensions + "]"

        dimensions = urllib.parse.quote(dimensions, safe='')

        url = baseURL + series + "/GeoArea/" + country_code + "/DataSlice?dimensions=" + dimensions + ""

    else:

        url = baseURL + series + "/GeoArea/" + country_code + "/DataSlice"
        
    #----------------------------------------------------------------
    # Make a GET request using the http object that you just created 
    #----------------------------------------------------------------

    response = http.request('GET', url)

    data = json.loads(response.data.decode('UTF-8'))
    data = data['dimensions']
    
    for d in data[:]:
        if numeric_part(d['value']) is None:
            data.remove(d)
    
    if len(data) > 0:

        for d in data:
            d['year'] = d.pop('timePeriodStart')
            
            if 'Reporting Type' in d.keys():
                del(d['Reporting Type'])
            

    
    return data

In [22]:
get_fact_data(fact_builder[1],'484')

[{'value': '2.8', 'Age': '15+', 'Sex': 'BOTHSEX', 'year': '2010'},
 {'value': '3.7', 'Age': '15+', 'Sex': 'BOTHSEX', 'year': '2004'},
 {'value': '6.8', 'Age': '15+', 'Sex': 'BOTHSEX', 'year': '2000'},
 {'value': '4.8', 'Age': '15+', 'Sex': 'BOTHSEX', 'year': '2002'},
 {'value': '2.8', 'Age': '15+', 'Sex': 'BOTHSEX', 'year': '2008'},
 {'value': '2.2', 'Age': '15+', 'Sex': 'BOTHSEX', 'year': '2012'},
 {'value': '4.5', 'Age': '15+', 'Sex': 'BOTHSEX', 'year': '2005'},
 {'value': '2.2', 'Age': '15+', 'Sex': 'BOTHSEX', 'year': '2006'},
 {'value': '2.7', 'Age': '15+', 'Sex': 'BOTHSEX', 'year': '2014'},
 {'value': '1.4', 'Age': '15+', 'Sex': 'BOTHSEX', 'year': '2016'}]

#### Create vector of values

In [23]:
def data_values(data):
        
    years = []
    values = []
    values_numeric_part = []
    values_alpha_part = []
    values_is_censored = []
    
    
    for d in data:
    
        years.append(d['year'])
        values.append(d['value'])
        values_numeric_part.append(numeric_part(d['value']))
        values_alpha_part.append(non_numeric_part(d['value']))
        

    
    for i in range(len(values)):
        values[i] = values[i].replace("<", "less than ")
        values[i] = values[i].replace(">", "more than ")
        
    for i in range(len(values_alpha_part)):
        
        if values_alpha_part[i] != '':
            values_is_censored.append(True)
        else:
            values_is_censored.append(False)

    
    for i in range(len(values_numeric_part)):
        if values_numeric_part[i] is not None:
            if values_numeric_part[i] <0.99 :
                values_numeric_part[i] = round_KFM(values_numeric_part[i],2)
            if values_numeric_part[i] >= 0.99 and values_numeric_part[i] <= 10:
                values_numeric_part[i] = round_KFM(values_numeric_part[i],1)
            if values_numeric_part[i] >10 :
                values_numeric_part[i] = round_KFM(values_numeric_part[i],0)
                
    return years, values, values_numeric_part, values_alpha_part, values_is_censored



In [24]:
data_values(get_fact_data(fact_builder[1],'484'))

(['2010',
  '2004',
  '2000',
  '2002',
  '2008',
  '2012',
  '2005',
  '2006',
  '2014',
  '2016'],
 ['2.8', '3.7', '6.8', '4.8', '2.8', '2.2', '4.5', '2.2', '2.7', '1.4'],
 [2.8, 3.7, 6.8, 4.8, 2.8, 2.2, 4.5, 2.2, 2.7, 1.4],
 ['', '', '', '', '', '', '', '', '', ''],
 [False, False, False, False, False, False, False, False, False, False])

In [ ]:
count_country = 0

for this_country in countryArray:
    
    if this_country['CountryProfile'] != '1':
        continue
        
    count_country += 1
            
    country_profile = {}
    
    country_profile['release'] = release 

    country_code = str(this_country['geoAreaCode'])
    country_name = this_country['geoAreaName']
    
    country_profile['country_code'] = country_code
    country_profile['country_name'] = country_name
    country_profile['X'] = this_country['X']
    country_profile['Y'] = this_country['Y']
    
    print("Building country profile for ", country_name, " - (", count_country, ")")
    
    count_fact = 0
    
    goals = []
    
    
    for g in indicator_framework:
        
        goal = {}
        
        goal['goalCode'] = g['code']
    
        targets = []
        
        for t in g['targets']:
            
            target = {}
            
            target['targetCode'] = t['code']
        
            indicators = []
                
            for i in t['indicators']:
                
                indicator = {}
                indicator['indicatorCode'] = i['code']
                
                facts = []
                
                for s in i['series']:
                    
                    if s['release'] == release:
                
                        for this_fact in fact_builder:
                        
                            if this_fact['SeriesCode'] != s['code']:
                                continue

                            
                            if this_fact['Profile.Series'] != '1':
                                continue
                            
                            count_fact += 1

                            fact = {}

                            print(" -- ", country_name, " - (", count_country, "): Adding fact - (", count_fact, ") for series ", s['code'], "of goal ", g['code'])

                            fact['seriesCode'] = s['code']
                            fact['seriesTitle'] = s['description']
                            fact['hub'] = this_fact['HubItemId']

                            fact['slice_dimensions'] = get_slice_dimensions(this_fact)

                            #----------------------------------------------------------------
                            # Main fact calculation 
                            #----------------------------------------------------------------

                            years, values, values_numeric_part, values_alpha_part, values_is_censored =  data_values(get_fact_data(this_fact, country_code))

                            n = None      # number of observations available
                            y_min = None    # first year available
                            y_max = None    # most recent year available

                            value_y_min = None   # data value in the first year available
                            value_y_max = None   # data value in the most recent year available

                            value_y_min_num = None   # data value in the first year available
                            value_y_max_num = None   # data value in the most recent year available


                            value_median = None
                            dif_first_last = None
                            value_y_max_is_censored = None

                            prog = ""
                            prog_10 = ""
                            prog_12 = ""
                            prog_15 = ""
                            prog_mmr_min = ""
                            prog_mmr_max = ""

                            fact_text = ""

                            if(len(values)>0):

                                n = len(values)      # number of observations available
                                y_min = min(years)    # first year available
                                y_max = max(years)    # most recent year available

                                value_y_min = values[years.index(min(years))]   # data value in the first year available
                                value_y_max = values[years.index(max(years))]   # data value in the most recent year available

                                value_y_min_num = values_numeric_part[years.index(min(years))]   # data value in the first year available
                                value_y_max_num = values_numeric_part[years.index(max(years))]   # data value in the most recent year available

                                value_median = statistics.median(values_numeric_part)
                                dif_first_last = abs(values_numeric_part[years.index(min(years))] \
                                                     - values_numeric_part[years.index(max(years))])

                                value_y_max_is_censored = values_is_censored[years.index(max(years))]


                                #--------------------------------------------------------
                                #Information about status of progress: decline/increase?
                                #---------------------------------------------------------

                                if (value_y_min_num > value_y_max_num):
                                    prog = this_fact['Down'] 
                                    # Example: "declined from", "down from"
                                elif (value_y_min_num < value_y_max_num):
                                    prog = this_fact['Up']
                                    # Example: "increased from", "up from"
                                else:
                                    prog = ""

                                #print(prog)

                                #--------
                                # prog.15
                                #--------
                                if value_y_min_num < 0.01:
                                     prog_15 = "nearly no coverage"
                                else:
                                    prog_15 = str(value_y_min_num) + this_fact['unit1']

                                #--------
                                # prog.10
                                #--------

                                if value_y_max_num < 10:
                                    prog_10 = this_fact['Down']
                                else:
                                    prog_10 = this_fact['Up']

                                #--------
                                # prog.12
                                #--------

                                if value_y_max_num > 0:
                                    prog_12 = this_fact['Up'] 
                                else:
                                    prog_12 = this_fact['Down']


                                #-------------
                                # prog_mmr_min
                                #-------------
                                if value_y_min_num > 1: 
                                    prog_mmr_min = "deaths"
                                else:
                                    prog_mmr_min = "death"

                                #-------------
                                # prog_mmr_max
                                #-------------
                                if value_y_max_num > 1: 
                                    prog_mmr_max = "deaths" 
                                else:
                                    prog_mmr_max = "death"

                                #print("min = ",value_y_min_num)
                                #print("max = ",value_y_max_num)
                                #print("prog = ",prog)
                                #print(prog_15)
                                #print(prog_10)
                                #print(prog_12)
                                #print(prog_mmr_min)
                                #print(prog_mmr_max)


                                #--------------------------------------------------------------------

                                fact_text = ""

                                condition1 = dif_first_last >= 0.05 * abs(value_y_max_num) 
                                condition2 = not value_y_max_is_censored
                                condition3 = value_y_max_num >= .25*value_median
                                condition4 = int(y_min) < 2010
                                condition5 = n > 1

                                conditions = condition1 and condition2 and condition3 and condition4 and condition5

                                #print("condition1 = ", condition1)
                                #print("condition2 = ", condition2)
                                #print("condition3 = ", condition3)
                                #print("condition4 = ", condition4)
                                #print("condition5 = ", condition5)
                                #print("conditions = ", conditions)


                                if this_fact['Text.type'] =='1':
                                    if conditions:
                                        fact_text = this_fact['DA3.1'] + prog + "<span class='fact-value'>" + str(value_y_min) + "</span>" + " <span class='fact-unit'>" + this_fact['unit1']  + "</span>" + " in " +  "<span class='fact-year'>" + str(y_min) + "</span>" + " to " + "<span class='fact-value'>" + str(value_y_max) + this_fact['unit1']+ "</span> " +  " in " +  " <span class='fact-year'>" + str(y_max)  + "</span>" + "."
                                        fact_values = [ str(value_y_min), str(value_y_max)]
                                        fact_units =  [ this_fact['unit1'], this_fact['unit1']]
                                        fact_years =  [ str(y_min), str(y_max)]
                                    else:
                                        fact_text = this_fact['DA2.1'] +  "<span class='fact-value'>" + str(value_y_max) + "</span>" + " <span class='fact-unit'>" + this_fact['unit1'] + "</span>"  + " in " + "<span class='fact-year'>" + str(y_max)  + "</span>" + "."
                                        fact_values = [ str(value_y_max)]
                                        fact_units = [ this_fact['unit1']]
                                        fact_years =  [ str(y_max)]

                                elif this_fact['Text.type'] == '2':
                                    fact_text = "In " +  "<span class='fact-year'>" + y_max  + "</span>" + ", " +  "<span class='fact-value'>" + str(value_y_max) + "</span>" + " <span class='fact-unit'>" + this_fact['unit1'] + "</span> " + this_fact['DA2.1']
                                    fact_values = [ str(value_y_max)]
                                    fact_units = [ this_fact['unit1']]
                                    fact_years =  [ str(y_max)]

                                elif this_fact['Text.type'] == '3':
                                    fact_text = "In " +  "<span class='fact-year'> " + y_max +  "</span> " + ", " + this_fact['DA2.1'] +   "<span class='fact-value'> " + str(value_y_max) + "</span>" + " <span class='fact-unit'>" + this_fact['unit1'] +  "</span> " + this_fact['DA2.2']
                                    fact_values = [ str(value_y_max)]
                                    fact_units = [ this_fact['unit1']]
                                    fact_years =  [ str(y_max)]

                                elif this_fact['Text.type'] == '4':
                                    fact_text = this_fact['DA2.1']+  "<span class='fact-value'>"  + str(value_y_max) +  "</span> " + " in " + "<span class='fact-year'> " + y_max +  "</span> " + ", meaning "  + str(float(value_y_max) * 100) +  this_fact['DA2.2'] + "." 
                                    fact_values = [ str(value_y_max)]
                                    fact_units = [ this_fact['unit1']]
                                    fact_years =  [ str(y_max)]

                                elif this_fact['Text.type'] == '7':
                                    fact_text = this_fact['DA2.1'] +  "<span class='fact-value'> " + str(value_y_max) +  "</span> " + " <span class='fact-unit'>" + this_fact['unit1']+  "</span> "  + " in "+  "<span class='fact-year'> "  + str(y_max)+  "</span> "  + "."
                                    fact_values = [ str(value_y_max)]
                                    fact_units = [ this_fact['unit1']]
                                    fact_years =  [ str(y_max)]

                                elif this_fact['Text.type'] == '8':
                                    fact_text = "In "+  "<span class='fact-year'> "  + y_max+  "</span> "  + ", " +  "<span class='fact-value'> " + str(value_y_max)+  "</span> "  + " <span class='fact-unit'>" + this_fact['unit1'] +  "</span> "+ this_fact['DA2.1'] + "."
                                    fact_values = [ str(value_y_max)]
                                    fact_units = [ this_fact['unit1']]
                                    fact_years =  [ str(y_max)]

                                elif this_fact['Text.type'] == '9':
                                    if conditions:
                                        fact_text = "In "+  "<span class='fact-year'> " + y_max +  "</span> "+ ", " + this_fact['DA2.1'] +  "<span class='fact-value'> "+ str(value_y_max) +  "</span> " + " <span class='fact-unit'>" + this_fact['unit1']+  "</span> " + ", " + prog +  "<span class='fact-value'> "+ str(value_y_min) +  "</span> " + " <span class='fact-unit'>" + this_fact['unit2'] +  "</span> " + " in "+  "<span class='fact-year'> " + y_min +  "</span> "
                                        fact_values = [ str(value_y_min) + this_fact['unit1'], str(value_y_max)+ this_fact['unit1']]
                                        fact_years =  [ str(y_min), str(y_max)]
                                    else:
                                        fact_text = "In "+  "<span class='fact-year'> " + y_max+  "</span> " + ", " + this_fact['DA2.1'] +  "<span class='fact-value'> "+ str(value_y_max) +  "</span> "  + " <span class='fact-unit'>" + this_fact['unit1']+  "</span> " + "."
                                        fact_values = [ str(value_y_max)]
                                        fact_units = [ this_fact['unit1']]
                                        fact_years =  [ str(y_max)]

                                elif this_fact['Text.type'] == '10':
                                    fact_text =  "In " +  "<span class='fact-year'> "+ y_max +  "</span> "+ ", " +  this_fact['DA2.1'] +  "<span class='fact-value'> "+ str(value_y_max) +  "</span> "  + " <span class='fact-unit'>" + this_fact['unit1']+  "</span> " + ". " + prog_10 
                                    fact_values = [ str(value_y_max)]
                                    fact_units = [ this_fact['unit1']]
                                    fact_years =  [ str(y_max)]

                                elif this_fact['Text.type'] == '11':
                                    fact_text = "In "+  "<span class='fact-year'> " + y_max+  "</span> " + ", " +  this_fact['DA2.1']+  "<span class='fact-value'> " + str(value_y_max) +  "</span> "  + " <span class='fact-unit'>" + this_fact['unit1'] +  "</span> "+ ". "
                                    fact_values = [ str(value_y_max)]
                                    fact_units = [ this_fact['unit1']]
                                    fact_years =  [ str(y_max)]

                                elif this_fact['Text.type'] == '12':
                                    fact_text = "As of " +  "<span class='fact-year'> "+ y_max +  "</span> "+ ", " + country_name + prog_12 + "."
                                    fact_values = [ str(value_y_max)]
                                    fact_units = [ this_fact['unit1']]
                                    fact_years =  [ str(y_max)]

                                elif this_fact['Text.type'] == '13':
                                    fact_text =  "In " +  "<span class='fact-year'> "+ y_max+  "</span> " + ", " +  this_fact['DA2.1']+  "<span class='fact-value'> " + str(value_y_max) +  "</span> " + " <span class='fact-unit'>" + this_fact['unit1'] +  "</span> "+ this_fact['DA2.2'] + "."
                                    fact_values = [ str(value_y_max)]
                                    fact_units = [ this_fact['unit1']]
                                    fact_years =  [ str(y_max)]

                                elif this_fact['Text.type'] == '14':
                                    if conditions:
                                        fact_text = this_fact['DA3.1'] + prog +   "<span class='fact-value'>" + str(value_y_min) +  "</span> "  + " <span class='fact-unit'>" + this_fact['unit1'] +  "</span> " + " in " +  "<span class='fact-year'> "+ y_min+  "</span> " + " to "+  "<span class='fact-value'> " + str(value_y_max) +  "</span> "  + " <span class='fact-unit'>"  + this_fact['unit1'] +  "</span> "+ " in " +  "<span class='fact-year'> " + y_max+  "</span> " + "."
                                        fact_values = [ str(value_y_min), str(value_y_max)]
                                        fact_units = [ this_fact['unit1'], this_fact['unit1']]
                                        fact_years =  [ str(y_min), str(y_max)]
                                    else:
                                        fact_text = this_fact['DA2.1'] +  "<span class='fact-value'> "+ str(value_y_max)+  "</span> " + " <span class='fact-unit'>" + prog_mmr_max + this_fact['unit1'] +  "</span> " + " in "+  "<span class='fact-year'> " + str(y_max) +  "</span> "+ "."
                                        fact_values = [ str(value_y_max) + prog_mmr_max]
                                        fact_units = [ this_fact['unit1']]
                                        fact_years =  [ str(y_max)]

                                elif this_fact['Text.type'] == '15':
                                    if conditions:
                                        fact_text =  "In "+  "<span class='fact-year'> " + y_max +  "</span> "+ ", " +  this_fact['DA2.1'] +  "<span class='fact-value'> "+ str(value_y_max) +  "</span> " + " <span class='fact-unit'>" + this_fact['unit1'] +  "</span> " + "," + prog + prog_15 + " in "+  "<span class='fact-year'> " + y_min +  "</span> "+ "."
                                        fact_values = [ str(value_y_min), str(value_y_max)]
                                        fact_units = [ this_fact['unit1'], this_fact['unit1']]
                                        fact_years =  [ str(y_min), str(y_max)]
                                    else:
                                        fact_text =  "In "+  "<span class='fact-year'> " + y_max+  "</span> " + ", " +  this_fact['DA2.1'] +  "<span class='fact-value'> "+ str(value_y_max)+  "</span> "  + " <span class='fact-unit'>" + this_fact['unit1'] +  "</span> "+ "."
                                        fact_values = [ str(value_y_max)]
                                        fact_values = [ this_fact['unit1']]
                                        fact_years =  [ str(y_max)]


                                fact['text_type'] = this_fact['Text.type']
                                fact['fact_text'] = fact_text
                                fact['fact_values'] = fact_values
                                fact['fact_units'] = fact_units
                                fact['fact_years'] = fact_years
                                fact['data_years'] = years
                                fact['data_values'] = values
                                fact['data_is_censored'] = values_is_censored
                                fact['data_numeric_part'] = values_numeric_part
                                
                                if this_fact['Text.type'] == '10':
                                    fact['preferred_visualization'] = 'threshold'
                                elif this_fact['Text.type'] == '12':
                                    fact['preferred_visualization'] = 'boolean'
                                elif(len(values)>3):
                                    fact['preferred_visualization'] = 'time_series'
                                else:
                                    fact['preferred_visualization'] = 'singleton'
                                

                                #=======================  
                                facts.append(fact)

                if len(facts)>0:    
                    indicator['facts'] = facts
                    indicators.append(indicator)

            if len(indicators)>0:
                target['indicators'] = indicators
                targets.append(target)
        
        if len(targets)>0:
            goal['targets'] = targets
            goals.append(goal)
        
    country_profile['goals'] = goals
    

                   
    #=======================

    #with open('resolutions.json', 'w') as outfile:
    with open("country profile " + country_code + " " + country_name + ".json", 'w') as outfile:
        json.dump(country_profile, outfile, indent=4 )    

    #with open('resolutions.json', 'w') as outfile:

    
    
            


Building country profile for  Afghanistan  - ( 1 )
 --  Afghanistan  - ( 1 ): Adding fact - ( 1 ) for series  SI_POV_DAY1 of goal  1
 --  Afghanistan  - ( 1 ): Adding fact - ( 2 ) for series  SI_POV_EMP1 of goal  1
 --  Afghanistan  - ( 1 ): Adding fact - ( 3 ) for series  SI_POV_NAHC of goal  1
 --  Afghanistan  - ( 1 ): Adding fact - ( 4 ) for series  SG_DSR_LEGREG of goal  1
 --  Afghanistan  - ( 1 ): Adding fact - ( 5 ) for series  SN_ITK_DEFC of goal  2
 --  Afghanistan  - ( 1 ): Adding fact - ( 6 ) for series  SH_STA_STUNT of goal  2
 --  Afghanistan  - ( 1 ): Adding fact - ( 7 ) for series  SH_STA_OVRWGT of goal  2
 --  Afghanistan  - ( 1 ): Adding fact - ( 8 ) for series  SH_STA_MMR of goal  3
 --  Afghanistan  - ( 1 ): Adding fact - ( 9 ) for series  SH_DYN_MORT of goal  3
 --  Afghanistan  - ( 1 ): Adding fact - ( 10 ) for series  SH_HIV_INCD of goal  3
 --  Afghanistan  - ( 1 ): Adding fact - ( 11 ) for series  SH_DTH_NCOM of goal  3
 --  Afghanistan  - ( 1 ): Adding fact - 